In [5]:
%pip install pandas morfessor 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\tanma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import morfessor
import platform


os = platform.system()

if os == 'Windows':
  training_tsv = 'data\eng.sentence.train.tsv'  #path to the training tsv file
  training_txt = 'data\\training_morfessor.txt' #path to the txt file
  input_tsv = 'data\eng.sentence.test.gold.tsv'  #path to the input tsv file
  output_tsv = 'outputs\eng.sentence.test.morfessor_guess.tsv'  #path tot the output tsv file
else:
  training_tsv = 'data/eng.sentence.train.tsv'
  training_txt = 'data/training_morfessor.txt'
  input_tsv = 'data/eng.sentence.test.gold.tsv'
  output_tsv = 'outputs/eng.sentence.test.morfessor_guess.tsv'


io = morfessor.MorfessorIO()
model = morfessor.BaselineModel()


df = pd.read_csv(training_tsv, sep='\t', header=None)
data = df[0].astype(str)  #data frame with the sentences

with open(training_txt, 'w', encoding='utf-8') as f:  #write training data into txt file
  for entry in data:
    f.write(entry + '\n')



# model.max_morph_length = 5
training_data = io.read_corpus_file(training_txt)
model.load_data(training_data)
model.train_batch()


df_guess = pd.read_csv(input_tsv, sep='\t', header=None)
data_guess = df_guess[0].astype(str)
i = 0

#sengment on sentence
'''
for entry in data_guess:
  sent = ''
  list_sent = model.viterbi_segment(entry)[0] #list of segmented sentence
  j = 0
  length = len(list_sent)
  for word in list_sent:
    if j != length - 1:
      sent += (word + ' @@')
      j+=1
    else :
      sent += word
  df_guess[1][i] = sent
  i+=1
'''

#segment on words

for entry in data_guess:
  sent = ''
  j = 0
  words = entry.split() #list of words
  length_sent = len(words)  #number of words

  for word in words:
    list_word = model.viterbi_segment(word)[0]
    k = 0
    length_word = len(list_word)  #number of morphs

    for morph in list_word:
      if k != length_word - 1:  #not last morph
        sent += (morph + ' @@')
        k+=1
      else: #last morph
        if j != length_sent - 1:
          sent += (morph + ' ')
        else :
          sent +=morph
    j+=1

  df_guess[1][i] = sent
  i+=1

df_guess.to_csv(output_tsv, sep='\t', header=None, index = False)

...........................................................
...........................................................
...........................................................
